In [54]:
import torch
import gym

In [55]:
env = gym.make('CartPole-v0')

In [56]:
nn = torch.nn.Sequential(
    torch.nn.Linear(4, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, env.action_space.n),
    torch.nn.Softmax(dim=-1)
)
optim = torch.optim.SGD(nn.parameters(), lr = 1e-3)

In [57]:
obs = torch.tensor(env.reset(), dtype=torch.float)
done = False
Actions, States, Rewards = [], [], []

In [58]:
while not done:
    probs = nn(obs)
    dist = torch.distributions.Categorical(probs=probs)
    action = dist.sample().item() 
    obs_, rew, done, _ = env.step(action)

    Actions.append(torch.tensor(action, dtype=torch.int))
    States.append(obs)
    Rewards.append(rew)

    obs = torch.tensor(obs_, dtype=torch.float)

In [59]:
y = 0.01
DiscountedReturns = []
for t in range(len(Rewards)):
    G = 0.0
    for k, r in enumerate(Rewards[t:]):
        G += (y**k) * r
    DiscountedReturns.append(G)
    

In [60]:
for State, Action, G in zip(States, Actions, DiscountedReturns):
    probs = nn(State)
    dist = torch.distributions.Categorical(probs=probs)
    log_prob = dist.log_prob(Action)

    loss = -log_prob*G

    optim.zero_grad()
    loss.backward()
    optim.step()